# Import Packages

In [1]:
# Other
import cv2
import numpy as np
from statistics import mode
from keras.models import load_model
# Local
from helper import get_labels
from helper import detect_faces
from helper import draw_text
from helper import draw_bounding_box
from helper import apply_offsets
from helper import load_detection_model
from helper import preprocess_input

Using TensorFlow backend.


# Model Paths

In [2]:
# assign variables to model path
face_detection_path = 'models/detection/haarcascade_frontalface_default.xml'
emotion_rec_path = 'models/emotion/XCEPTION_107-0.66.hdf5'
emotion_labels = get_labels()

# Bounding Boxes

In [3]:
frame_window = 10
emotion_offsets = (20, 40)

# Load Models

In [ ]:
# loading models
face_detection = load_detection_model(face_detection_path)
emotion_rec = load_model(emotion_rec_path, compile=False)

Instructions for updating:
Colocations handled automatically by placer.


In [ ]:
# getting input model shapes for displaying results
emotion_target_size = emotion_rec.input_shape[1:3]

In [ ]:
# starting lists for calculating modes
emotion_window = []

# Start Video Streaming for Emotion Detection

In [ ]:
# starting video streaming
cv2.namedWindow('Camera_Emotion_Detection') # set window name
video_capture = cv2.VideoCapture(0) # assign default webcame as video source
while True:
    bgr_img = video_capture.read()[1] # read the frame
    gray_img = cv2.cvtColor(bgr_img, cv2.COLOR_BGR2GRAY) # convert BGR to GRAY
    rgb_img = cv2.cvtColor(bgr_img, cv2.COLOR_BGR2RGB) # convert BGR to RGB
    faces = detect_faces(face_detection, gray_img) # detect faces using Haar

    for face_coordinates in faces: # for each face detected

        x1, x2, y1, y2 = apply_offsets(face_coordinates, emotion_offsets) # apply face and emotion coordinates
        gray_face = gray_img[y1:y2, x1:x2] # set face coordinates on grayscale image
        try:
            gray_face = cv2.resize(gray_face, (emotion_target_size)) # resize the image to emotion image size
        except:
            continue

        gray_face = preprocess_input(gray_face, True) # convert pixel values to between -1 and 1
        gray_face = np.expand_dims(gray_face, 0) # add space after each pixel coordinate
        gray_face = np.expand_dims(gray_face, -1) # add new line after each pixel coordinate
        emotion_prediction = emotion_rec.predict(gray_face) # use model to predict face emotion
        emotion_probability = np.max(emotion_prediction) # assign the mostly like emotion values
        emotion_label_arg = np.argmax(emotion_prediction) # assign most occuring emotion value
        emotion_text = emotion_labels[emotion_label_arg] # # assign corresponding emotion text
        emotion_window.append(emotion_text) # append the emotion text to array

        if len(emotion_window) > frame_window: # if number of appended emotions is greater than 10
            emotion_window.pop(0) # remove the oldest predicted emotion
        try:
            emotion_mode = mode(emotion_window) # create array of most commonly occuring emotions
        except:
            continue

        # setup emotion colors
        if emotion_text == 'angry':
            color = emotion_probability * np.asarray((255, 0, 0))
        elif emotion_text == 'sad':
            color = emotion_probability * np.asarray((0, 0, 255))
        elif emotion_text == 'happy':
            color = emotion_probability * np.asarray((255, 255, 0))
        elif emotion_text == 'surprise':
            color = emotion_probability * np.asarray((0, 255, 255))
        else:
            color = emotion_probability * np.asarray((0, 255, 0))

        color = color.astype(int)
        color = color.tolist()

        draw_bounding_box(face_coordinates, rgb_img, color) # draw face detection
        draw_text(face_coordinates, rgb_img, emotion_mode, color, 0, -45, 1, 1) # draw the emotion text

    bgr_img = cv2.cvtColor(rgb_img, cv2.COLOR_RGB2BGR)
    cv2.imshow('window_frame', bgr_img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break